# PageRank

On calcule PageRank avec la méthode de puissance :

$$p^{(k+1)} = T \cdot p^{(k)}$$

Règles du modèle (énoncé) :

- T[i, j] = probabilité d'aller de la page j vers la page i (colonne = page de départ)
- Si une page n'a aucun lien sortant (sink) : prochaine page uniforme (1/N)
- Arrêt quand :

$$\frac{||T p^{(k)} - p^{(k)}||_1}{||p^{(k)}||_1} \leq \epsilon$$

In [46]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

## 1) Lire les CSV
- names.csv : colonne Name
- edges.csv : colonnes FromNode, ToNode (IDs 1-based)


In [47]:
liste_pages = pd.read_csv("names.csv")     # Name
liste_liens = pd.read_csv("edges.csv")       # FromNode, ToNode

nombre_pages = len(liste_pages)

pages_depart = liste_liens["FromNode"].to_numpy() - 1   # j
pages_arrivee = liste_liens["ToNode"].to_numpy() - 1     # i

nombre_pages, len(liste_liens)


(199903, 10722190)

## 2) Degrés sortants + sinks

In [48]:
nbr_liens_page = np.bincount(pages_depart, minlength=nombre_pages)
impasses = (nbr_liens_page == 0)

int(impasses.sum())


0

## 3) Construction de la matrice de transition $T$ (Sparse)

Le dataset étant volumineux, on utilise une **matrice creuse (Sparse Matrix)** de SciPy pour optimiser la mémoire et la rapidité des calculs.

* **Format utilisé** : `csr_matrix` (Compressed Sparse Row), idéal pour les multiplications matrice-vecteur.
* **Contenu de $T$** : Si un lien existe de la page $j$ vers la page $i$, l'élément $T_{ij}$ reçoit la probabilité $$\frac{1}{outdeg[j]}$$
* **Gestion des "Sinks"** : Les pages sans liens sortants ne sont pas stockées dans la matrice creuse pour économiser de l'espace. Leur masse est redistribuée dynamiquement lors de l'itération pour simuler un saut uniforme vers n'importe quelle page du réseau.

In [49]:
# Construction de la partie "liens" de la matrice de transition
# Chaque lien j -> i reçoit une probabilité de 1 / outdeg[j]
proba_liens_page = 1.0 / nbr_liens_page[pages_depart]
T = csr_matrix((proba_liens_page, (pages_arrivee, pages_depart)), shape=(nombre_pages, nombre_pages))

## 4) Méthode de puissance + critère d'arrêt

On cherche à calculer le vecteur de PageRank $\vec{p}^{(\infty)}$ par itérations successives :

$$\vec{p}^{(k+1)} = T \cdot \vec{p}^{(k)}$$

L'algorithme s'arrête lorsque la variation relative entre deux itérations devient inférieure à un seuil $\epsilon$ :

$$\frac{||T\vec{p}^{(k)} - \vec{p}^{(k)}||_1}{||\vec{p}^{(k)}||_1} \leq \epsilon$$

Dans cette version, le calcul $T \cdot \vec{p}^{(k)}$ est effectué en combinant la multiplication par la matrice creuse et l'ajout manuel de la contribution uniforme des pages "sinks".

In [50]:
eps = 1e-8
k = 0
p_k = np.ones(nombre_pages) / nombre_pages
d = 0.85

while True:
    # p_kplus1 = (Partie creuse des liens) * p_k + (Partie uniforme des impasses)
    # L'opérateur @ déclenche la multiplication optimisée de SciPy
    p_kplus1 = d*(T @ p_k + p_k[impasses].sum() / nombre_pages) + (1 - d) / nombre_pages

    err = np.abs(p_kplus1 - p_k).sum() / np.abs(p_k).sum()
    p_k = p_kplus1
    k += 1

    if err <= eps:
        break

k, err, p.sum()

(49, np.float64(8.44661695525518e-09), np.float64(0.9999999999999989))

## 5) Top pages

In [51]:
top_k = 20
idx = np.argsort(-p)[:top_k]

pd.DataFrame({
    "rank": np.arange(1, top_k + 1),
    "node_id": idx + 1,
    "name": names["Name"].iloc[idx].to_numpy(),
    "pagerank": p[idx],
})


,rank,node_id,name,pagerank
0,1,112356,United States,0.002491
1,2,168241,United Kingdom,0.001390
2,3,138128,World War II,0.001131
3,4,184958,Latin,0.001084
4,5,60041,France,0.001077
5,6,138420,Germany,0.000919
6,7,49148,English language,0.000839
7,8,149853,China,0.000797
8,9,151511,Canada,0.000791
9,10,145591,India,0.000789


## 6) Recherche basique (titre contient le mot-clé)

On filtre par nom et trie par score PageRank

In [ ]:
recherche = "New York"
k = 10

recherche = recherche.lower()
mask = names["Name"].str.lower().str.contains(recherche, na=False).to_numpy()
idx = np.where(mask)[0]
idx = idx[np.argsort(-p[idx])][:k]

results = pd.DataFrame({
    "rank": np.arange(1, len(idx) + 1),
    "node_id": idx + 1,
    "name": names["Name"].iloc[idx].to_numpy(),
    "pagerank": p[idx],
})
results


,rank,node_id,name,pagerank
0,1,113505,New York City,0.000496
1,2,21033,New York,0.000364
2,3,113621,The New York Times,0.000320
3,4,125654,New York Stock Exchange,0.000057
4,5,113402,New York University,0.000052
5,6,96841,The New Yorker,0.000044
6,7,13273,"Buffalo, New York",0.000042
7,8,197152,New York Yankees,0.000035
8,9,21032,"Albany, New York",0.000028
9,10,13152,"Rochester, New York",0.000025
